In [1]:
import pdfplumber
import pandas as pd

In [7]:
# Ruta del archivo PDF
ruta_pdf = r"C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\2024\745723.pdf"
#ruta_pdf = r"C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\2024\746022.pdf"

# Lista para guardar líneas de texto
lineas = []

# Leer el PDF línea por línea
with pdfplumber.open(ruta_pdf) as pdf:
    for pagina in pdf.pages:
        texto = pagina.extract_text()
        if texto:
            lineas.extend(texto.split('\n'))

# Crear DataFrame
df = pd.DataFrame(lineas, columns=["Texto"])


In [4]:
# Mostrar hasta 100 filas en la salida
pd.set_option('display.max_rows', 100)

# Mostrar hasta 100 columnas (opcional)
pd.set_option('display.max_columns', 100)

# Mostrar hasta 200 caracteres por columna
pd.set_option('display.max_colwidth', 200)

In [3]:
import pdfplumber
import pandas as pd
import re
import os
from datetime import datetime
from PyPDF2 import PdfReader

In [4]:
def extraer_fecha_linea(linea):
    """
    Extrae fecha según patrones:
    - 'Ruedas del 2 al 5 de enero de 2024'
    - 'Rueda del 4 de enero de 2024'
    - 'Anexo A Análisis de opciones 31/01/2022'
    """
    # Patron 1: Ruedas del 2 al 5 de enero de 2024
    patron_intervalo = r"Ruedas del \d+ al (\d+) de (\w+) de (\d{4})"
    match = re.match(patron_intervalo, linea)
    if match:
        dia, mes_str, anio = match.groups()
        mes_num = convertir_mes_a_numero(mes_str.lower())
        if mes_num:
            return datetime(int(anio), mes_num, int(dia)).strftime("%Y-%m-%d")
    
    # Patron 2: Rueda del 4 de enero de 2024
    patron_simple = r"Rueda del (\d+) de (\w+) de (\d{4})"
    match = re.match(patron_simple, linea)
    if match:
        dia, mes_str, anio = match.groups()
        mes_num = convertir_mes_a_numero(mes_str.lower())
        if mes_num:
            return datetime(int(anio), mes_num, int(dia)).strftime("%Y-%m-%d")

    # Patron 3: Anexo A Análisis de opciones 31/01/2022
    patron_anexo = r"Anexo A Análisis de opciones (\d{2}/\d{2}/\d{4})"
    match = re.match(patron_anexo, linea)
    if match:
        fecha_str = match.group(1)
        return datetime.strptime(fecha_str, "%d/%m/%Y").strftime("%Y-%m-%d")
    
    return None

def convertir_mes_a_numero(mes_str):
    meses = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5,
        'junio': 6, 'julio': 7, 'agosto': 8, 'septiembre': 9,
        'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }
    return meses.get(mes_str)

def extraer_pdf_con_fecha(ruta_pdf, errores):
    lineas = []
    fecha_encontrada = None
    try:
        print(f"Intentando abrir PDF: {ruta_pdf}") # ¡Línea crucial para depurar!
        with pdfplumber.open(ruta_pdf) as pdf:
            for pagina in pdf.pages:
                texto = pagina.extract_text()
                if texto:
                    for linea in texto.split('\n'):
                        if not fecha_encontrada:
                            fecha = extraer_fecha_linea(linea)
                            if fecha:
                                fecha_encontrada = fecha
                        lineas.append(linea)

        if not fecha_encontrada:
            errores.append(ruta_pdf)
            fecha_encontrada = pd.NA

        return pd.DataFrame({
            "Fecha": [fecha_encontrada] * len(lineas),
            "Texto": lineas
        })

    except Exception as e:
        # Captura cualquier error que ocurra al abrir o procesar el PDF
        error_msg = f"Error al procesar el archivo '{ruta_pdf}': {e}"
        print(f"🔴 {error_msg}") # Imprime el error para verlo inmediatamente
        errores.append(error_msg)
        return pd.DataFrame(columns=["Fecha", "Texto"]) # Devuelve un DataFrame vacío para no romper pd.concat

def extraer_mes_carpeta(ruta_carpeta_mes):
    """
    Procesa todos los PDFs de una carpeta, devuelve un DataFrame único,
    y guarda lista de archivos sin fecha.
    """
    df_total = pd.DataFrame(columns=["Fecha", "Texto"])
    errores = []

    for root, dirs, files in os.walk(ruta_carpeta_mes):
        for file in files:
            if file.lower().endswith('.pdf'):
                ruta_pdf = os.path.join(root, file)
                print(f"Procesando: {ruta_pdf}")
                df_pdf = extraer_pdf_con_fecha(ruta_pdf, errores)
                df_total = pd.concat([df_total, df_pdf], ignore_index=True)

    # Guardar errores en un txt
    if errores:
        ruta_errores = os.path.join(ruta_carpeta_mes, "informes_sin_fecha.txt")
        with open(ruta_errores, "w", encoding="utf-8") as f:
            for path in errores:
                f.write(f"{path}\n")
        print(f"⚠️ Archivos sin fecha: {len(errores)}. Ver {ruta_errores}")

    return df_total



In [5]:
def procesar_ruta_abuelo(ruta_abuelo):
    """
    Recorre recursivamente todas las subcarpetas desde la ruta_abuelo,
    aplica extraer_mes_carpeta en cada carpeta que tenga PDFs,
    y guarda un único archivo parquet consolidado.
    """
    df_total = pd.DataFrame(columns=["Fecha", "Texto"])
    
    for root, dirs, files in os.walk(ruta_abuelo):
        # Verificamos si la carpeta actual contiene PDFs
        pdfs = [f for f in files if f.lower().endswith('.pdf')]
        if pdfs:
            print(f"📁 Procesando carpeta con PDFs: {root}")
            df_parcial = extraer_mes_carpeta(root)
            df_total = pd.concat([df_total, df_parcial], ignore_index=True)

    # Guardamos el resultado completo en un solo archivo
    ruta_salida = os.path.join(ruta_abuelo, "datos_todos_los_pdf.parquet")
    df_total.to_parquet(ruta_salida, index=False)
    print(f"✅ Archivo parquet consolidado guardado en:\n{ruta_salida}")



In [9]:
procesar_ruta_abuelo(r"C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones")


📁 Procesando carpeta con PDFs: C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\2016\1
Procesando: C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\2016\1\11.pdf
Intentando abrir PDF: C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\2016\1\11.pdf
Procesando: C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\2016\1\12.pdf
Intentando abrir PDF: C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\2016\1\12.pdf
Procesando: C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\2016\1\13.pdf
Intentando abrir PDF: C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\2016\1\13.pdf
Procesando: C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesi

In [2]:
# Define la ruta completa de tu archivo Parquet
ruta_archivo_parquet = r"C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\datos_todos_los_pdf.parquet"


# Lee el archivo Parquet en un DataFrame
df = pd.read_parquet(ruta_archivo_parquet)